# CSCI 3155 Spring 2025
# Recitation Week 3
### Topic: Inductive Definitions, Grammars, and Syntax Trees
This week we will be doing pattern matching and inductive data structures, and get some practice using them.

# Pattern Matching
Pattern matching is a control structure we will use many times in this class, especially when dealing with inductive structures.
Let's see some of the ways we can use pattern matching.

# Exercise 1
Let's write a function with pattern matching.
The function `listOutput` takes in type `List[Int]` and returns a type `String`

For a empty list output `"Empty list"`

For a list with a single element output `"The number is $n$"`

#### The first two cases are given, write the cases below

For a list that has two elements output `"The numbers are $n1$ and $n2$"`

For a list that has more than two elements output `"Multiple numbers, with head being $n$"`


In [2]:
def listOutput(myList: List[Int]): String = {
    myList match {
        case Nil => "Empty list"
        case h :: Nil => s"The number is $h"
        // Begin solution
        case h :: t :: Nil => s"The numbers are $h and $t"
        case h :: _ => "Multiple numbers, with head being " + h
        // End solution
    }
}

defined function listOutput

In [3]:
val list1Test = Nil
val list2Test = 3 :: list1Test
val list3Test = 2 :: 5 :: list2Test
val list4Test = 10 :: list2Test
assert(listOutput(list3Test) == "Multiple numbers, with head being 2")
assert(listOutput(list4Test) == "The numbers are 10 and 3")

list1Test: Nil.type = List()
list2Test: List[Int] = List(3)
list3Test: List[Int] = List(2, 5, 3)
list4Test: List[Int] = List(10, 3)

# Inductive data structures
Pattern matching is particularly useful for interacting with inductive data structures.
Consider the following example, a list of integers.  (This is similar to how `List`s are implemented in Scala.)

$$\begin{array}{ccccc}
\textbf{NumList} & \rightarrow & Empty \\ &\ |\  & Cons(\textbf{Num}, \textbf{NumList}) \\
\textbf{Num} & \rightarrow & 0 \ |\ 1\ |\ 2\ |\ 3\ |\ 4\ |\ \cdots \\
\end{array}$$

In [4]:
sealed trait NumList

case object Empty extends NumList
case class Cons(hd: Int, tl: NumList) extends NumList

defined trait NumList
defined object Empty
defined class Cons

In [5]:
// example lists
// using the :: (cons operator) and Nil cases
val scalaList = 131 :: 3155 :: Nil
// this is equivalent to using the List constructor
scalaList == List(131, 3155)

// the same list using our NumList data structure instead of the built in one
val myNumList = Cons(131, Cons(3155, Empty))

scalaList: List[Int] = List(131, 3155)
res4_1: Boolean = true
myNumList: Cons = Cons(131, Cons(3155, Empty))

# Exercise 2
For this exercise, consider a simple language representing arithmetic expressions.
Given the grammar below, write the corresponding Scala code (assume $\textbf{Num}$ can be represented by a Scala `Int` as in $\textbf{NumList}$).

$$\begin{array}{ccccccccc}
\textbf{Expr} & \rightarrow & Const(\textbf{Num}) \\ &\ |\  & Plus(\textbf{Expr}, \textbf{Expr}) \\ &\ |\  & Minus(\textbf{Expr}, \textbf{Expr}) \\ &\ |\  & Times(\textbf{Expr}, \textbf{Expr}) \\
\textbf{Num} & \rightarrow & 0 \ |\ 1\ |\ 2\ |\ 3\ |\ 4\ |\ \cdots \\
\end{array}$$

In [1]:
// Begin Solution
sealed trait Expr

case class Const(n: Int) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Times(e1: Expr, e2: Expr) extends Expr
// end solution

defined trait Expr
defined class Const
defined class Plus
defined class Minus
defined class Times

In [3]:
// expressions using our new grammer
val myExpr0 = Const(0)
val myExpr1 = Plus(Const(500), Const(131)) // 500 + 131
val myExpr2 = Times(myExpr1, Minus(Const(105), Const(100))) // (500+131)*(105-100)

myExpr0: Const = Const(n = 0)
myExpr1: Plus = Plus(e1 = Const(n = 500), e2 = Const(n = 131))
myExpr2: Times = Times(
  e1 = Plus(e1 = Const(n = 500), e2 = Const(n = 131)),
  e2 = Minus(e1 = Const(n = 105), e2 = Const(n = 100))
)

In [4]:
// Come up with your own example
val myExpr3 = Const(42) // Const(42) or just anything else

myExpr3: Const = Const(n = 42)

Now that we have our implementation of the grammar, we can write functions using this implementation.
We will often want to pattern match on the different cases of our grammar when writing these functions, as in the example below.

#### In this exercise, you will write a function that takes in an AST expression and returns a list containing all the constants

Examples. 


$$\begin{array}{ccc}
Const(42)  & \textit{--> constVals(  ) -->} & [42] \\ 
Plus(Const(500), Const(131)) & \textit{--> constVals(  ) -->} & [500, 131]\\
Times(Plus(Const(500), Const(131)), Minus(Const(105), Const(100))) & \textit{--> constVals(  ) -->} & [500, 131, 105, 100]\\
\end{array}$$




In [5]:
// Get the constants in an expression
def constVals(e: Expr): List[Int] = {
    e match {
        case Const(n) => n :: Nil // or List(n)
        case Plus(e1, e2) => constVals(e1) ++ constVals(e2)
        case Minus(e1, e2) => constVals(e1) ++ constVals(e2)
        case Times(e1, e2) => constVals(e1) ++ constVals(e2)
    }
}

defined function constVals

In [6]:
constVals(myExpr0)
constVals(myExpr2)
constVals(myExpr3)

res6_0: List[Int] = List(0)
res6_1: List[Int] = List(500, 131, 105, 100)
res6_2: List[Int] = List(42)

# Exercise 3
## Eval
Implement a function eval to compute the value of an expression represented as an Abstract Syntax Tree (AST). The AST nodes are defined by the following grammar:

$$\begin{array}{ccccccccc}
\textbf{Expr} & \rightarrow & Const(\textbf{Num}) \\ &\ |\  & Plus(\textbf{Expr}, \textbf{Expr}) \\ &\ |\  & Minus(\textbf{Expr}, \textbf{Expr}) \\ &\ |\  & Times(\textbf{Expr}, \textbf{Expr}) \\
\end{array}$$

Write cases for `Const`, `Plus`, `Minus`, and `Times`

In [11]:
def eval(e: Expr): Int = {
    // Begin Solution
    e match {
        case Const(n) => n
        case Plus (e1, e2) => eval(e1) + eval(e2)
        case Minus(e1, e2) => eval(e1) - eval(e2)
        case Times(e1, e2) => eval(e1) * eval(e2)
    }
    // End Solution
}

defined function eval

In [12]:
// test cases
eval(Const(42)) == 42
eval(myExpr0) == 0 // 0
eval(myExpr1) == 631 // 500 + 131
eval(myExpr2) == 3155 // (500+131)*(105-100)
eval(myExpr3)

res11_0: Boolean = true
res11_1: Boolean = true
res11_2: Boolean = true
res11_3: Boolean = true
res11_4: Int = 42

## Bonus


### A. Enhancing `Expr` Trait
How would you add division or identifiers/variables to the Expr trait?

We will look at this more in the future, but thinking through this is a good way to practice.
(Hint: start by expanding the grammar and adding another case for the Expr trait.)

## Additional Practice

### A. Pattern matching for Tokens

Let's write a function with pattern matching to simplify a `WordExpr` that is defined using a inductively defined data-type `Tokens`. 

`WordExpr` is defined inductively as follows
$$\begin{array}{ccccccccc}
\color{blue}{\textbf{WordExpr}} & \rightarrow & \color{blue}{NullExpr} \\ 
&\ |\  & \color{blue}{ConsExpr}(\color{green}{\textbf{Token}}, \color{blue}{\textbf{WordExpr}}) \\ 
&\ |\  & \color{blue}{SeqExpr}( \color{blue}{\textbf{WordExpr}}, \color{blue}{\textbf{WordExpr}}) \\ 
\end{array}$$


where `Token` is described by the following grammar
$$\begin{array}{ccccccccc}
\color{green}{\textbf{Token}} & \rightarrow & \color{green}{EmptyTok} \\ 
&\ |\  & \color{green}{ChrTok}(\textbf{Char}) \\ 
&\ |\  & \color{green}{StrTok}(\textbf{String}) \\ 
\end{array}$$


We would like to define a function $\textbf{flattenWordExpr}: \textbf{WordExpr} \rightarrow \textbf{Token}$ that simplifies the inductive structure into the simplest Token that can capture the information contained in it. This function can be thought of as effectively computing the **concatenation of all characters and string** tokens contained within the WordExpr. 

For example: 
$$\begin{array}{ccc}
\color{blue}{NullExpr} & \textit{--> flattenWordExpr(  ) -->} & \color{green}{EmptyTok} \\ 
 \color{blue}{ConsExpr}( \color{green}{ChrTok}(`c`), \color{blue}{NullExpr}) & \textit{--> flattenWordExpr(  ) -->} & \color{green}{ChrTok}(`c`)\\
 \color{blue}{ConsExpr}(\color{green}{EmptyTok}, \color{blue}{ConsExpr}(\color{green}{ChrTok}(`c`),\color{blue}{NullExpr})) & \textit{--> flattenWordExpr(  ) -->} & \color{green}{ChrTok}(`c`)\\
 \color{blue}{SeqExpr}(\color{blue}{NullExpr}, \color{blue}{NullExpr}) & \textit{--> flattenWordExpr(  ) -->} & \color{green}{EmptyTok} \\
\color{blue}{SeqExpr}(\color{blue}{ConsExpr}( \color{green}{ChrTok}(`c`), \color{blue}{NullExpr}), \color{blue}{NullExpr}) & \textit{--> flattenWordExpr(  ) -->} & \color{green}{ChrTok}(`c`)\\
 \color{blue}{SeqExpr}(\color{blue}{NullExpr}, \color{blue}{ConsExpr}( \color{green}{ChrTok}(`c`), \color{blue}{NullExpr})) & \textit{--> flattenWordExpr(  ) -->}& \color{green}{ChrTok}(`c`) \\
\color{blue}{SeqExpr}(\color{blue}{ConsExpr}( \color{green}{ChrTok}(`c`), \color{blue}{NullExpr}), \color{blue}{ConsExpr}( \color{green}{ChrTok}(`c`), \color{blue}{NullExpr})) & \textit{--> flattenWordExpr(  ) -->} & \color{green}{StrTok}(\text{"cc"}) \\
\color{blue}{SeqExpr}(\color{blue}{ConsExpr}( \color{green}{StrTok}(\text{"cat"}), \color{blue}{NullExpr}), \color{blue}{ConsExpr}( \color{green}{StrTok}(\text{"dog"}), \color{blue}{NullExpr})) & \textit{--> flattenWordExpr(  ) -->} & \color{green}{StrTok}(\text{"catdog"})
\end{array}$$


In [13]:
sealed trait Token
case object EmptyTok extends Token
case class ChrTok(c: Char) extends Token
case class StrTok(s: String) extends Token



defined trait Token
defined object EmptyTok
defined class ChrTok
defined class StrTok

In [14]:
sealed trait WordExpr
case object NullExpr extends WordExpr
case class ConsExpr(t:Token, w:WordExpr) extends WordExpr
case class SeqExpr(w1:WordExpr, w2:WordExpr) extends WordExpr

defined trait WordExpr
defined object NullExpr
defined class ConsExpr
defined class SeqExpr

In [15]:
//BEGIN SOLUTION
def flattenWordExpr(w: WordExpr): Token = w match {
    case NullExpr => EmptyTok
    case ConsExpr(t,w) => {
            val t2 = flattenWordExpr(w)
            (t,t2) match{
                case (EmptyTok,y:Token) => y
                case (x:Token, EmptyTok) => x
                case (ChrTok(a),ChrTok(b)) => StrTok(a.toString+b.toString)
                case (ChrTok(a),StrTok(b)) => StrTok(a.toString+b)
                case (StrTok(a),ChrTok(b)) => StrTok(a+b.toString)
                case (StrTok(a),StrTok(b)) => StrTok(a+b)
            }
        }
    case SeqExpr(w1:WordExpr, w2:WordExpr) => {
        val t1 = flattenWordExpr(w1)
        val t2 = flattenWordExpr(w2)
        
        (t1,t2) match{
            case (EmptyTok,y:Token) => y
            case (x:Token, EmptyTok) => x
            case (ChrTok(a),ChrTok(b)) => StrTok(a.toString+b.toString)
            case (ChrTok(a),StrTok(b)) => StrTok(a.toString+b)
            case (StrTok(a),ChrTok(b)) => StrTok(a+b.toString)
            case (StrTok(a),StrTok(b)) => StrTok(a+b)
        }//(t1,t2)
    }// SeqExpr(w1,w2)
}
//END SOLUTION

defined function flattenWordExpr

##### Evaluation of function: **flattenWordExpr()**

In [16]:
//NullExpr --> flattenWordExpr(  ) --> EmptyTok
val e1:WordExpr = NullExpr
assert(flattenWordExpr(e1) == EmptyTok, "test1")

val e2:WordExpr = SeqExpr( ConsExpr(ChrTok('c'),NullExpr), ConsExpr(StrTok("at"),NullExpr))
assert(flattenWordExpr(e2) == StrTok("cat"), "test2")

val e3:WordExpr = SeqExpr( ConsExpr(StrTok("ca"),NullExpr), ConsExpr(ChrTok('t'),NullExpr))
assert(flattenWordExpr(e3) == StrTok("cat"), "test3")

//Cons( ChrTok('c'), NullExpr) --> flattenWordExpr(  ) --> ChrTok('c')
val e4:WordExpr = ConsExpr( ChrTok('c'), NullExpr)
assert(flattenWordExpr(e4) == ChrTok('c'), "test4")

//Cons(EmptyTok, Cons(ChrTok('c'),NullExpr)) --> flattenWordExpr(  ) --> ChrTok('c')
val e5:WordExpr = ConsExpr(EmptyTok, ConsExpr(ChrTok('c'),NullExpr)) 
assert(flattenWordExpr(e5) == ChrTok('c'), "test5")

//Sequence(NullExpr, NullExpr) --> flattenWordExpr(  ) --> EmptyTok
val e6:WordExpr = SeqExpr(NullExpr, NullExpr)
assert(flattenWordExpr(e6) == EmptyTok, "test6")

//5. Sequence(Cons( ChrTok('c'), NullExpr), NullExpr) --> flattenWordExpr(  ) --> ChrTok('c')
val e7:WordExpr = SeqExpr(ConsExpr( ChrTok('c'), NullExpr), NullExpr)
assert(flattenWordExpr(e7) == ChrTok('c'), "test7")

//6. Sequence(NullExpr, Cons( ChrTok('c'), NullExpr)) --> flattenWordExpr(  ) --> ChrTok('c')
val e8:WordExpr = SeqExpr(NullExpr, ConsExpr( ChrTok('c'), NullExpr))
assert(flattenWordExpr(e8) == ChrTok('c'), "test8")

//7. Sequence(Cons( ChrTok('c'), NullExpr), Cons( ChrTok('c'), NullExpr)) --> flattenWordExpr(  ) --> StrTok("cc")
val e9:WordExpr = SeqExpr(ConsExpr( ChrTok('c'), NullExpr), ConsExpr( ChrTok('c'), NullExpr))
assert(flattenWordExpr(e9) == StrTok("cc"), "test9")

//8. Sequence(Cons( StrTok("cat"), NullExpr), Cons( StrTok("dog"), NullExpr)) --> flattenWordExpr(  ) --> StrTok("catdog")
val e10:WordExpr = SeqExpr(ConsExpr( StrTok("cat"), NullExpr), ConsExpr( StrTok("dog"), NullExpr)) 
assert(flattenWordExpr(e10) == StrTok("catdog"), "test10")


e1: WordExpr = NullExpr
e2: WordExpr = SeqExpr(
  ConsExpr(ChrTok('c'), NullExpr),
  ConsExpr(StrTok("at"), NullExpr)
)
e3: WordExpr = SeqExpr(
  ConsExpr(StrTok("ca"), NullExpr),
  ConsExpr(ChrTok('t'), NullExpr)
)
e4: WordExpr = ConsExpr(ChrTok('c'), NullExpr)
e5: WordExpr = ConsExpr(EmptyTok, ConsExpr(ChrTok('c'), NullExpr))
e6: WordExpr = SeqExpr(NullExpr, NullExpr)
e7: WordExpr = SeqExpr(ConsExpr(ChrTok('c'), NullExpr), NullExpr)
e8: WordExpr = SeqExpr(NullExpr, ConsExpr(ChrTok('c'), NullExpr))
e9: WordExpr = SeqExpr(
  ConsExpr(ChrTok('c'), NullExpr),
  ConsExpr(ChrTok('c'), NullExpr)
)
e10: WordExpr = SeqExpr(
  ConsExpr(StrTok("cat"), NullExpr),
  ConsExpr(StrTok("dog"), NullExpr)
)